In [1]:

import pandas as pd
import pydantic
import openai
import os
import anthropic
import instructor
import json
import plotly.graph_objects as go
from __future__ import annotations
import logging 
import typing


from rich.console import Console
console = Console()

# logging.basicConfig(level=logging.DEBUG)



In [2]:
class dimension(pydantic.BaseModel):
    dimension_name: str
    description: str
    children: list[dimension]

class response(pydantic.BaseModel):
    dimensions: list[dimension]


In [12]:
class generate_hierarchy:

    # initialize class variables
    def __init__(
            self,
            top_level:str,
            max_iterations,
            client:Literal['openai', 'anthropic'],
            top_levels_separate:bool = False
            ) -> None:
        with open('input_domains/' + str(top_level) + '_domains.json') as f:
            self.input = json.load(f)
        if top_levels_separate:
            with open('prompts/' + str(top_level) + '_separate_prompt.txt') as f:
                self.prompt = f.read()
        else:
            with open('prompts/' + str(top_level) + '_prompt.txt') as f:
                self.prompt = f.read()
        self.max_iterations = max_iterations
        self.top_level = top_level
        self.client = client
        self.top_levels_separate = top_levels_separate
        self.results = []
        self.dimension_names = []
        self.dimension_descriptions = []

    # call api to generate hierarchy
    def api_call(self, prompt, input, domain_name: typing.Optional[str]):
        max_iterations = self.max_iterations
        domain_name = domain_name
        input = input
        # openai gpt-4o
        if self.client == 'openai':
            for i in range(1, self.max_iterations + 1):
                print(i)
                client=instructor.from_openai(openai.OpenAI())
                message = client.chat.completions.create(
                    model='gpt-4o',
                    response_model=response,
                    messages=[
                        {'role':'user',
                        'content':eval(prompt)},
                    ]
                )
                iterations_completed = i
                current_model = [x.model_dump() for x in message.dimensions]
                if input == current_model:
                    print('finished on iteration ', i-1)
                    break
                if iterations_completed == self.max_iterations:
                    print('completed all ', self.max_iterations, ' iterations')
                input = current_model
        # anthropic claude 3.5 sonnet
        if self.client == 'anthropic':
            region = 'us-west-2'
            for i in range(1, self.max_iterations + 1):
                print(i)
                client = instructor.from_anthropic(anthropic.AnthropicBedrock(
                    aws_access_key = os.environ.get('AWS_ACCESS_KEY_ID'),
                    aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY'),
                    aws_region=region,
                ))

                message = client.messages.create(
                    model='anthropic.claude-3-5-sonnet-20241022-v2:0',
                    response_model=response,
                    max_tokens=80000,
                    messages=[
                        {'role':'user',
                        'content':eval(prompt)},
                        ],
                    )
                iterations_completed = i 
                current_model = [x.model_dump() for x in message.dimensions]
                print(current_model)
                if input == current_model:
                    print('finished on iteration ', i)
                    break
                if iterations_completed == self.max_iterations:
                    print('completed all ', self.max_iterations, ' iterations')
                input = current_model

        return current_model, iterations_completed

    # parse model to format as table
    def parse_node(self, node, hierarchy):
        if len(node['children']) == 0:
            self.results.append(hierarchy + [node['dimension_name']])
            if node['dimension_name'] not in self.dimension_names:
                self.dimension_names.append(node['dimension_name'])
            if node['description'] not in self.dimension_descriptions:
                self.dimension_descriptions.append(node['description'])   
        else:
            self.results.append(hierarchy + [node['dimension_name']])
            for child in node['children']:
                child_hierarchy = hierarchy.copy()
                child_hierarchy.append(node['dimension_name'])
                if node['dimension_name'] not in self.dimension_names:
                    self.dimension_names.append(node['dimension_name'])
                if node['description'] not in self.dimension_descriptions:
                    self.dimension_descriptions.append(node['description']) 
                self.parse_node(child, child_hierarchy)

    # generate and save the hierarchy 
    def run(self):
        if self.top_levels_separate:
            model = []
            iterations_completed = 0
            for domain in self.input:
                domain_name = domain['dimension_name']
                domain_model, domain_iterations_completed = self.api_call(self.prompt, domain, domain_name)
                model.append(domain_model[0])
                if domain_iterations_completed > iterations_completed:
                    iterations_completed = domain_iterations_completed
        else:
            model, iterations_completed = self.api_call(self.prompt,self.input)
        for domain in model:
            self.parse_node(domain, []) 
        description_dict = dict(zip(self.dimension_names, self.dimension_descriptions))
        for row in self.results:
            print(row)
            dimension = row[-1]
            print(dimension)
            description = description_dict[dimension]
            if len(row) < (iterations_completed + 1):
                row.extend([pd.NA] * ((iterations_completed + 1) - len(row)))
            row.append(description)
        print(row)
        result_df = pd.DataFrame(self.results)
        result_df.columns = [f'level_{i}' for i in range(1, len(result_df.columns))] + ['description']
        # save model as json and csv
        with open('results/' + str(self.top_level) + '_hierarchy.json', 'w') as f:
            json.dump(model, f)
        result_df = result_df.dropna(how='all', axis=1).fillna('')
        result_df.to_csv('results/' + str(self.top_level) + '_' + str(self.client) + '_hierarchy.csv', index=False)
        return model, result_df


In [13]:
model, result_df = generate_hierarchy(top_level='generated', max_iterations=5, client = 'anthropic', top_levels_separate=True).run()

1
[{'dimension_name': 'Cognitive Function', 'description': 'The mental processes involved in acquiring, processing, and utilizing information, including thinking, reasoning, learning, and memory.', 'children': [{'dimension_name': 'Memory', 'description': 'The ability to encode, store, retain, and recall information and experiences.', 'children': []}, {'dimension_name': 'Executive Function', 'description': 'Higher-order cognitive processes that enable goal-directed behavior, including planning, decision-making, and mental flexibility.', 'children': []}, {'dimension_name': 'Processing Speed', 'description': 'The efficiency and speed with which cognitive tasks are performed and information is processed.', 'children': []}, {'dimension_name': 'Language Processing', 'description': 'The cognitive aspects of understanding and using language, including comprehension and production.', 'children': []}, {'dimension_name': 'Visual-Spatial Processing', 'description': 'The ability to perceive, analyz

In [5]:
result_df

,level_1,level_2,level_3,level_4,level_5,description
0,Cognitive Function,,,,,Core mental abilities and processes that enabl...
1,Cognitive Function,Memory,,,,"The ability to encode, store, and retrieve inf..."
2,Cognitive Function,Memory,Working Memory,,,The ability to temporarily hold and manipulate...
3,Cognitive Function,Memory,Working Memory,Verbal Working Memory,,The ability to temporarily hold and manipulate...
4,Cognitive Function,Memory,Working Memory,Verbal Working Memory,Phonological Loop,The component responsible for temporarily stor...
...,...,...,...,...,...,...
655,Self-Concept and Identity,Identity Development,Identity Commitment,Goal Commitment,,Personal memories and recollections that contr...
656,Self-Concept and Identity,Identity Development,Identity Integration,,,Significant changes and turning points that ha...
657,Self-Concept and Identity,Identity Development,Identity Integration,Identity Coherence,,"Goals, dreams, and plans that contribute to on..."
658,Self-Concept and Identity,Identity Development,Identity Integration,Identity Flexibility,,How individuals understand and express themsel...


In [43]:
i = 1
region = 'us-east-1'
with open('RDoC_domains.json') as f:
    input = json.load(f)
with open('RDoC_prompt.txt') as f:
    prompt = f.read()
client = anthropic.AnthropicBedrock(
        aws_access_key = os.environ.get('AWS_ACCESS_KEY_ID'),
        aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY'),
        aws_region=region,
    )

message = client.messages.create(
        model='anthropic.claude-3-5-sonnet-20240620-v1:0',
        max_tokens=8000,
        messages=[
            {'role':'user',
            'content':eval(prompt)},
            ]
        )
iterations_completed = i 
print(message)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/maya.roberts/code/aimless/aimless-env/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:anthropic._base_client:Request options: {'method': 'post', 'url': '/model/anthropic.claude-3-5-sonnet-20240620-v1:0/invoke', 'timeout': 600, 'files': None, 'json_data': {'max_tokens': 8000, 'messages': [{'role': 'user', 'content': 'We are creating a hierarchical model of neuropsychological concepts. This model will be ultimately used as rating scales to assess function in specific dimensions of neuropsychology. \nEach dimension of functioning should be neutral and applicable to the general population, not only disorders. Extreme presentations of a dimension may be related to a cognitive or psychological disorder. \nEach dimension should also relate to an evidence-based scientific concept or construct for the field of neuropsychology and child development.\nEach dimensio

Message(id='msg_bdrk_01SY5QJcGik5ZyswR4zsNVx5', content=[TextBlock(text='Here\'s the updated model with one level of subdivision added to each root dimension:\n\n[\n  {\n    "dimension_name": "Negative Valence Systems",\n    "description": "Negative Valence Systems are primarily responsible for responses to aversive situations or context, such as fear, anxiety, and loss.",\n    "children": [\n      {\n        "dimension_name": "Acute Threat (Fear)",\n        "description": "Activation of the fear system in response to present or imminent threat.",\n        "children": []\n      },\n      {\n        "dimension_name": "Potential Threat (Anxiety)",\n        "description": "Activation of a system of responses to potential harm or threat.",\n        "children": []\n      },\n      {\n        "dimension_name": "Sustained Threat",\n        "description": "An aversive emotional state caused by prolonged (i.e., weeks to months) exposure to internal and/or external condition(s), state(s), or sti

In [31]:
with open('RDoC_domains.json') as f:
    input = json.load(f)
with open('RDoC_prompt.txt') as f:
    prompt = f.read()


for i in range(i, 10):
    client = anthropic.AnthropicBedrock(
            aws_access_key = os.environ.get('AWS_ACCESS_KEY_ID'),
            aws_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY'),
            aws_region='us-east-1',
        )

    message = client.messages.create(
        model='anthropic.claude-3-5-sonnet-20240620-v1:0',
        max_tokens=8000,
        messages=[
            {'role':'user',
            'content':eval(prompt)},
            
            ]
        )

In [37]:
input

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x105dc9bd0>>

In [39]:
with open('RDoC_prompt.txt') as f:
    prompt = f.read()
with open('RDoC_domains.json') as f:
    input = json.load(f)
i = 4
print(eval(prompt))

We are creating a hierarchical model of neuropsychological concepts. This model will be ultimately used as rating scales to assess function in specific dimensions of neuropsychology. 
Each dimension of functioning should be neutral and applicable to the general population, not only disorders. Extreme presentations of a dimension may be related to a cognitive or psychological disorder. 
Each dimension should also relate to an evidence-based scientific concept or construct for the field of neuropsychology and child development.
Each dimension must have a "parent" dimension (unless they are "root" dimensions) and zero, one or more "child" dimensions, illustrating the notion of sub-components or special cases of dimensions.

The “root” dimensions, or highest level psychological domains, should be the following:
    1. Negative Valence Systems: primarily responsible for responses to aversive situations or context, such as fear, anxiety, and loss.
    2. Positive Valence Systems: primarily r

In [21]:
result_df.to_csv('RDoC_hierarchy.csv', index=False)

In [7]:
pd.unique(result_df['level_8'])

array([''], dtype=object)

In [7]:
import plotly.express as px
fig = px.treemap(
    names = dimensions,
    parents = parents,
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25), width=2000, height=2000)
fig.show()

NameError: name 'dimensions' is not defined

In [138]:




fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      color = "blue"
    ),
    link = dict(
      source = dimensions, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = parents,
      value = [1] * len(parents) ,
  ))])

fig.update_layout(width = 2000, height = 1000)
fig.show()